## Load Data

In [4]:
import pandas as pd

data = pd.read_csv("data.csv")
data = data["content"].drop_duplicates().dropna()[:37000]
print(data.info())

<class 'pandas.core.series.Series'>
Int64Index: 37000 entries, 0 to 37044
Series name: content
Non-Null Count  Dtype 
--------------  ----- 
37000 non-null  object
dtypes: object(1)
memory usage: 578.1+ KB
None


## Removing Stopwords

In [5]:
import nltk
nltk.download('stopwords')
nltk.download('words')

stop_words = set(nltk.corpus.stopwords.words('english'))
corp = set(nltk.corpus.words.words())

# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([word for word in textArr if word not in stop_words and word in corp])
    return rem_text

# remove stopwords from the text
data=data.apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /home/tawfik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/tawfik/nltk_data...
[nltk_data]   Package words is already up-to-date!


## Lemmatization

In [6]:
import spacy
from gensim import corpora

# nlp = en_core_web_sm.load()
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output
tokenized_reviews = lemmatization(data.tolist())

# print number of tokenization
from importlib.util import find_spec as isModule
if(isModule('humanize') != None):
      from humanize import intword
      print(intword(sum(len(x) for x in tokenized_reviews)), " Tokenizations")
else:
      print(sum(len(x) for x in tokenized_reviews), " Tokenizations")

dictionary = corpora.Dictionary(tokenized_reviews)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_reviews]

3.8 million  Tokenizations


## Model Training

In [7]:
import gensim
# Creating the object for LDA model using gensim library
Lda = gensim.models.LdaMulticore

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(corpus=doc_term_matrix, id2word=dictionary, num_topics=25, passes=10)
print("-"*10, "\n", "Topics\n", ldamodel.print_topics(num_words=10))

---------- 
 Topics
 [(12, '0.029*"company" + 0.015*"business" + 0.012*"border" + 0.011*"new" + 0.009*"mexican" + 0.008*"executive" + 0.008*"financial" + 0.007*"chief" + 0.007*"food" + 0.007*"money"'), (19, '0.036*"immigration" + 0.024*"illegal" + 0.020*"order" + 0.018*"executive" + 0.015*"government" + 0.013*"country" + 0.013*"border" + 0.012*"administration" + 0.012*"ban" + 0.011*"new"'), (5, '0.024*"white" + 0.019*"people" + 0.017*"black" + 0.013*"man" + 0.010*"gay" + 0.008*"social" + 0.007*"actor" + 0.007*"time" + 0.006*"film" + 0.005*"woman"'), (24, '0.053*"police" + 0.034*"gun" + 0.026*"people" + 0.014*"law" + 0.012*"violence" + 0.010*"enforcement" + 0.010*"city" + 0.009*"fire" + 0.009*"man" + 0.008*"protest"'), (11, '0.042*"percent" + 0.015*"poll" + 0.013*"game" + 0.013*"last" + 0.013*"first" + 0.012*"team" + 0.010*"second" + 0.009*"good" + 0.009*"time" + 0.008*"final"'), (10, '0.017*"political" + 0.014*"man" + 0.011*"government" + 0.010*"many" + 0.009*"people" + 0.007*"power" +

## Evaluation

In [8]:
import pyLDAvis
import pyLDAvis.gensim_models
%matplotlib inline

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary)
vis

/home/tawfik/.local/share/miniconda3/envs/nlp/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
18    -0.028400 -0.070150       1        1  7.523829
22     0.105170 -0.100857       2        1  7.423005
2     -0.095247  0.060147       3        1  6.999452
3     -0.105653 -0.017068       4        1  5.647912
15    -0.094992 -0.046615       5        1  5.400870
16     0.060745  0.149353       6        1  5.357178
21     0.012268 -0.046920       7        1  5.108217
6     -0.140561 -0.030436       8        1  4.624464
17    -0.002824 -0.013929       9        1  4.316253
9      0.015751 -0.081215      10        1  4.218237
1     -0.006648  0.102047      11        1  4.176261
19    -0.117379  0.123004      12        1  4.148709
11    -0.028092 -0.115761      13        1  4.090690
12    -0.055016 -0.038208      14        1  3.858324
5      0.108200 -0.074055      15        1  3.670152
0      0.143006  0.156524      16        1  3.414599
24     0.108505  0.137462      17        1  3.113742
13    -0.103318 -0.066462      18        1  3.108647
8      0.036561  0.002306      19        1  2.616583
10     0.025106  0.005671      20        1  2.592875
23     0.003772  0.061044      21        1  2.295528
4     -0.135448  0.065170      22        1  1.871187
14    -0.004467  0.005657      23        1  1.825031
7      0.108370 -0.125948      24        1  1.491975
20     0.190589 -0.040761      25        1  1.106279, topic_info=             Term          Freq         Total Category  logprob  loglift
366        police  16040.000000  16040.000000  Default  30.0000  30.0000
358       percent  13251.000000  13251.000000  Default  29.0000  29.0000
141         black   7109.000000   7109.000000  Default  28.0000  28.0000
595   immigration   7150.000000   7150.000000  Default  27.0000  27.0000
1295     campaign  16902.000000  16902.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
344           old    182.106249   4870.948877  Topic25  -5.4401   1.2177
618          life    185.880988   6347.935635  Topic25  -5.4196   0.9734
108          time    217.925580  24429.101852  Topic25  -5.2606  -0.2152
71           many    214.055095  22055.606177  Topic25  -5.2785  -0.1309
243         first    169.401063  17906.379225  Topic25  -5.5124  -0.1565

[2107 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
16417     17  0.775751      abdoman
6820       9  0.055645   aboriginal
6820      20  0.890323   aboriginal
4674       1  0.026554     abortion
4674       2  0.001130     abortion
...      ...       ...          ...
5584      21  0.058511          zoo
5584      22  0.014628          zoo
5584      24  0.014628          zoo
5584      25  0.833778          zoo
24079      6  0.867266  zoroastrian

[17089 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 23, 3, 4, 16, 17, 22, 7, 18, 10, 2, 20, 12, 13, 6, 1, 25, 14, 9, 11, 24, 5, 15, 8, 21])

In [9]:
print('\nPerplexity: ', ldamodel.log_perplexity(doc_term_matrix,total_docs=len(data.index)))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=ldamodel, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/home/tawfik/.local/share/miniconda3/envs/nlp/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -7.699343065161069
